In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9415  0.9342  0.9362  0.9345  0.9398  0.9372  0.0029  
MAE (testset)     0.7444  0.7335  0.7391  0.7354  0.7398  0.7384  0.0038  
Fit time          3.76    3.91    3.78    3.81    3.76    3.81    0.06    
Test time         0.16    0.16    0.12    0.16    0.13    0.15    0.02    


{'fit_time': (3.7636237144470215,
  3.9143483638763428,
  3.783771276473999,
  3.8097941875457764,
  3.7584526538848877),
 'test_mae': array([0.74435065, 0.73349348, 0.73907652, 0.73541883, 0.73981748]),
 'test_rmse': array([0.941472  , 0.9342054 , 0.93618882, 0.93448088, 0.93978533]),
 'test_time': (0.15989947319030762,
  0.1583549976348877,
  0.1224522590637207,
  0.15810847282409668,
  0.12773489952087402)}